## Import ##

In [1]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch import optim
import os
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import transforms, CenterCrop, RandomCrop, ToTensor, Normalize, Compose
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import tqdm
from sklearn.metrics import accuracy_score

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Data ##

In [20]:
class MyData(Dataset):
    def __init__(self, path, size = 250):
        self.path = path
        self.file_list = os.listdir(self.path)
        self.size = size
    
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, idx):
        img = Image.open( os.path.join(self.path, self.file_list[idx] ) )
        cmp = Compose([CenterCrop(self.size), 
                       ToTensor(), 
                      ])
        l_str = str(self.file_list[idx])[:3]
        if (l_str == 'cat'):
            label = torch.Tensor([0, 1])
        else: 
            label = torch.Tensor([1, 0])
        return cmp(img), label

In [21]:
ds = MyData('A:\\My files\\Datasets\\dogs-vs-cats\\train\\train')

In [22]:
#set_ds = DataLoader(ds, shuffle = True, batch_size = 5)

In [34]:
train, test = torch.utils.data.random_split(ds, lengths = [int(round(len(ds)*0.7)), int(round(len(ds)*0.3))])
train, test = DataLoader(train, shuffle = True, batch_size = 20), DataLoader(test, shuffle = True, batch_size = 20)

## Net ##

In [44]:
class NN(nn.Module):
    def __init__(self,):
        super(NN, self).__init__()
        self.c1 = nn.Conv2d(3,6,1,1)
        self.mp1 = nn.MaxPool2d(3,3)
        self.c2 = nn.Conv2d(6,9,3,1)
        self.c3 = nn.Conv2d(9,20,3,1)
        self.c4 = nn.Conv2d(20,50,3,1)
        self.c5 = nn.Conv2d(50,100,1,1)
        self.ln1 = nn.Linear(400, 100) # 800
        self.lr = nn.LeakyReLU()
        self.ln2 = nn.Linear(100, 2)
        self.b1 = nn.BatchNorm2d(6)
        self.b2 = nn.BatchNorm2d(9)
        self.b3 = nn.BatchNorm2d(20)
        self.b4 = nn.BatchNorm2d(50)
        self.b5 = nn.BatchNorm2d(100)
        
        
        #self.skip_conv = nn.Conv2d(9,100,80,1)
        
    def forward(self, x):
        x = self.c1(x) # 250 250
        x = self.mp1(x) # 83 83
        x = self.b1(x)
        x = self.lr(x)
        #x2 = self.c2(x) # 81 81
        #x = self.mp1(x2) # 27 27
        x = self.c2(x) # 81 81
        x = self.mp1(x) # 27 27
        x = self.b2(x)
        x = self.lr(x)
        x = self.c3(x) # 24 24
        x = self.mp1(x) # 8 8
        x = self.b3(x)
        x = self.lr(x)
        x = self.c4(x) # 6 6
        x = self.mp1(x) # 2 2
        x = self.b4(x)
        x = self.lr(x)
        x = self.c5(x) # 2 2
        x = self.b5(x)
        x = self.lr(x)
        
        #sk_xo = self.skip_conv(x2)
        #x = torch.cat([x, sk_xo], dim = 1)
        
        x = x.view(x.size()[0], torch.prod(torch.tensor(x.size())[1:]) )
        x = self.lr(x)
        x = self.ln1(x)
        x = self.lr(x)
        x = self.ln2(x)
        return x

In [45]:
n = NN()

In [46]:
o = n(ds[0][0].view(1,3,250,250))

In [47]:
o

tensor([[-0.3485,  0.1472]], grad_fn=<AddmmBackward0>)

In [48]:
n.to(device)

NN(
  (c1): Conv2d(3, 6, kernel_size=(1, 1), stride=(1, 1))
  (mp1): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (c2): Conv2d(6, 9, kernel_size=(3, 3), stride=(1, 1))
  (c3): Conv2d(9, 20, kernel_size=(3, 3), stride=(1, 1))
  (c4): Conv2d(20, 50, kernel_size=(3, 3), stride=(1, 1))
  (c5): Conv2d(50, 100, kernel_size=(1, 1), stride=(1, 1))
  (ln1): Linear(in_features=400, out_features=100, bias=True)
  (lr): LeakyReLU(negative_slope=0.01)
  (ln2): Linear(in_features=100, out_features=2, bias=True)
  (b1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (b2): BatchNorm2d(9, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (b3): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (b4): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (b5): BatchNorm2d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [49]:
opt = optim.Adam(n.parameters())
ls = nn.BCEWithLogitsLoss().to(device)

## Train-loop ##

In [50]:
def train_net(net, data, opt, loss, epochs, device):
    for epoch in range(0, epochs, 1):
        for X,y in tqdm.tqdm_notebook(data):
            X = X.to(device)
            y = y.to(device)
            
            opt.zero_grad()
            o = net(X)
            ls = loss(o, y)
            ls.backward()
            opt.step()

In [51]:
train_net(n, train, opt, ls, 1, device)

c:\python\python36\tf\lib\site-packages\ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


  0%|          | 0/875 [00:00<?, ?it/s]

Test

In [54]:
def forward_pass(model, Val_loader):
    model.eval()
    y_s = []
    op_s = []
    with torch.no_grad():
        with tqdm.tqdm_notebook(Val_loader) as tq_val_data:
            for i, (X,y) in enumerate(tq_val_data):
                X = X.to(device)
                y = y
                
                op = model(X)
                y_s.append(y)
                op_s.append(op)
    return y_s, op_s

In [55]:
y, op = forward_pass(n, test)

c:\python\python36\tf\lib\site-packages\ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/375 [00:00<?, ?it/s]

In [56]:
y = np.array([lbl.cpu().detach().numpy() for lbl in y]).flatten()

In [57]:
op = np.array([ans.cpu().detach().numpy() for ans in op]).flatten()

In [58]:
op

array([-0.15415221,  0.17216821, -1.8181978 , ..., -0.6338313 ,
        0.4771213 , -0.5063829 ], dtype=float32)

In [59]:
op_ = np.where(op > 0.5, 1, 0)

In [60]:
accuracy_score(y, op_)

0.7148